In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel("./Ecommerce Sales Analysis.xlsx")
df.head(3)

,Row ID,Order ID,Year,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2013-152156,2013,2013-11-09,2013-11-12,Second Class,CG-12520,Claire Gute,Consumer,United States,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2013-152156,2013,2013-11-09,2013-11-12,Second Class,CG-12520,Claire Gute,Consumer,United States,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820
2,3,CA-2013-138688,2013,2013-06-13,2013-06-17,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.0,6.8714


In [3]:
df.shape

(9994, 22)

In [7]:
for col in df.columns:
    print(col)

Row ID
Order ID
Year
Order Date
Ship Date
Ship Mode
Customer ID
Customer Name
Segment
Country
City
State
Postal Code
Region
Product ID
Category
Sub-Category
Product Name
Sales
Quantity
Discount
Profit


In [1]:
import pandas as pd

# Sample Data
data = {
    'Year': [2020, 2021, 2022, 2023],
    'Sales': [1000, 1200, 1500, 1800]
}

df = pd.DataFrame(data)

# Calculate YoY Growth
df['YoY_Growth (%)'] = df['Sales'].pct_change() * 100

print(df)


   Year  Sales  YoY_Growth (%)
0  2020   1000             NaN
1  2021   1200            20.0
2  2022   1500            25.0
3  2023   1800            20.0


In [1]:
import pandas as pd
import psycopg2
import psycopg2.extras
import os

csv_files = [
    ('Ecommerce Sales Analysis.xlsx', 'ecommerce_sales')
]

folder_path = 'C:/Users/moham/Desktop/Data Analysis/2025 Data Analysis/Excel For Data Analysis/Project_1'

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INTEGER'
    elif pd.api.types.is_float_dtype(dtype):
        return 'REAL'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'TIMESTAMP'
    else:
        return 'TEXT'

try:
    conn = psycopg2.connect(
        host='localhost',
        user='postgres',
        password='mohammad',
        dbname='E-Commerce_2',
        port='5432'
    )
    cursor = conn.cursor()
    print("Connected to the database.")

    for file_name, table_name in csv_files:
        file_path = os.path.join(folder_path, file_name)

        df = pd.read_excel(file_path)

        df = df.where(pd.notnull(df), None)
        df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

        columns = ', '.join([f'"{col}" {get_sql_type(df[col].dtype)}' for col in df.columns])
        create_table_query = f'CREATE TABLE IF NOT EXISTS "{table_name}" ({columns})'
        cursor.execute(create_table_query)
        print(f"Table {table_name} created or already exists.")

        columns_str = ', '.join([f'"{col}"' for col in df.columns])
        placeholders = ', '.join(['%s'] * len(df.columns))
        insert_sql = f'INSERT INTO "{table_name}" ({columns_str}) VALUES ({placeholders})'
        data_tuples = [tuple(row) for row in df.to_numpy()]
        psycopg2.extras.execute_batch(cursor, insert_sql, data_tuples)

        conn.commit()
        print(f"Data from {file_name} inserted successfully into {table_name}.")

except psycopg2.Error as err:
    print(f"Error: {err}")
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    print("Database connection closed.")


Connected to the database.
Table ecommerce_sales created or already exists.
Data from Ecommerce Sales Analysis.xlsx inserted successfully into ecommerce_sales.
Database connection closed.


In [1]:
import psycopg2

try:
    # Connect to a different database (not the one you want to delete)
    conn = psycopg2.connect(
        host='localhost',
        user='postgres',
        password='mohammad',
        dbname='postgres',  # 👈 Not 'E-Commerce_2'
        port='5432'
    )
    conn.autocommit = True  # Required for DROP DATABASE
    cursor = conn.cursor()

    # Database you want to delete
    database_to_drop = 'E-Commerce_2'
    
    # Terminate all connections to the database
    cursor.execute(f"""
        SELECT pg_terminate_backend(pg_stat_activity.pid)
        FROM pg_stat_activity
        WHERE pg_stat_activity.datname = '{database_to_drop}'
        AND pid <> pg_backend_pid();
    """)
    
    # Now drop the database
    cursor.execute(f'DROP DATABASE IF EXISTS "{database_to_drop}"')
    print(f'Database "{database_to_drop}" has been successfully deleted.')

except psycopg2.Error as e:
    print(f'Error: {e}')
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()


Database "E-Commerce_2" has been successfully deleted.


In [2]:
import psycopg2

try:
    # Connect to the default 'postgres' database
    conn = psycopg2.connect(
        host='localhost',
        user='postgres',
        password='mohammad',
        dbname='postgres',  # Default database
        port='5432'
    )
    conn.autocommit = True  # Required for CREATE DATABASE
    cursor = conn.cursor()

    # Name of the database to create
    database_to_create = 'ECommerce'

    # Create the database
    cursor.execute(f'CREATE DATABASE "{database_to_create}"')
    print(f'Database "{database_to_create}" has been successfully created.')

except psycopg2.Error as e:
    print(f'Error: {e}')
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

Database "ECommerce" has been successfully created.


In [3]:
import pandas as pd
import psycopg2
import os

import psycopg2.extras

# File and table details
file_name = 'Ecommerce Sales Analysis.xlsx'
table_name = 'ecommerce_sales'
folder_path = 'C:/Users/moham/Desktop/Data Analysis/2025 Data Analysis/Excel For Data Analysis/Project_1'
file_path = os.path.join(folder_path, file_name)

# Function to determine SQL data types
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INTEGER'
    elif pd.api.types.is_float_dtype(dtype):
        return 'REAL'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'TIMESTAMP'
    else:
        return 'TEXT'

try:
    # Connect to the ECommerce database
    conn = psycopg2.connect(
        host='localhost',
        user='postgres',
        password='mohammad',
        dbname=database_to_create,  # Use the ECommerce database
        port='5432'
    )
    cursor = conn.cursor()
    print("Connected to the ECommerce database.")

    # Read the Excel file into a DataFrame
    df = pd.read_excel(file_path)

    # Clean column names and handle null values
    df = df.where(pd.notnull(df), None)
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Create the table if it doesn't exist
    columns = ', '.join([f'"{col}" {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS "{table_name}" ({columns})'
    cursor.execute(create_table_query)
    print(f"Table {table_name} created or already exists.")

    # Insert data into the table
    columns_str = ', '.join([f'"{col}"' for col in df.columns])
    placeholders = ', '.join(['%s'] * len(df.columns))
    insert_sql = f'INSERT INTO "{table_name}" ({columns_str}) VALUES ({placeholders})'
    data_tuples = [tuple(row) for row in df.to_numpy()]
    psycopg2.extras.execute_batch(cursor, insert_sql, data_tuples)

    conn.commit()
    print(f"Data from {file_name} inserted successfully into {table_name}.")

except psycopg2.Error as err:
    print(f"Error: {err}")
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    print("Database connection closed.")

Connected to the ECommerce database.
Table ecommerce_sales created or already exists.
Data from Ecommerce Sales Analysis.xlsx inserted successfully into ecommerce_sales.
Database connection closed.


In [4]:
df = pd.read_excel("./Ecommerce Sales Analysis.xlsx")
df.head(3)

,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,NaN,NaN,NaN
1,Row Labels,Sum of Sales,Sum of Profit
2,Jan,95365.9376,9199.27


In [5]:
import psycopg2

try:
    # Connect to the ECommerce database
    conn = psycopg2.connect(
        host='localhost',
        user='postgres',
        password='mohammad',
        dbname=database_to_create,  # Use the ECommerce database
        port='5432'
    )
    cursor = conn.cursor()

    # Drop the table
    cursor.execute(f'DROP TABLE IF EXISTS "{table_name}"')
    conn.commit()
    print(f'Table "{table_name}" has been successfully deleted from the database.')

except psycopg2.Error as e:
    print(f'Error: {e}')
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

Table "ecommerce_sales" has been successfully deleted from the database.


In [6]:
df = pd.read_csv("./ecommerce_sales_10k.csv")
df.head(3)

,Order ID,Order Date,Ship Date,Customer ID,Customer Name,Segment,City,State,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,ORD-96379d02,2025-02-20,2025-02-27,CUST-9902,Sandra Manning,Consumer,New York,California,Central,PROD-78727,Technology,Copiers,Copiers - Drive,477.54,8,0.2,47.65
1,ORD-c186dc48,2024-07-28,2024-08-02,CUST-3775,Sandra Lewis,Consumer,New York,Michigan,West,PROD-38771,Office Supplies,Art,Art - Choice,492.05,3,0.2,58.35
2,ORD-afd3e140,2025-02-17,2025-02-24,CUST-2583,Robert Moody,Corporate,Los Angeles,New York,Central,PROD-90836,Furniture,Furnishings,Furnishings - Power,534.12,10,0.3,70.81


In [7]:
df.columns

Index(['Order ID', 'Order Date', 'Ship Date', 'Customer ID', 'Customer Name',
       'Segment', 'City', 'State', 'Region', 'Product ID', 'Category',
       'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount',
       'Profit'],
      dtype='object')

In [8]:
import pandas as pd
import psycopg2

import psycopg2.extras

# File and table details
csv_file_path = './ecommerce_sales_10k.csv'
table_name_csv = 'ecommerce_sales_10k'

# Function to determine SQL data types
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INTEGER'
    elif pd.api.types.is_float_dtype(dtype):
        return 'REAL'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'TIMESTAMP'
    else:
        return 'TEXT'

try:
    # Connect to the ECommerce database
    conn = psycopg2.connect(
        host='localhost',
        user='postgres',
        password='mohammad',
        dbname=database_to_create,  # Use the ECommerce database
        port='5432'
    )
    cursor = conn.cursor()
    print("Connected to the ECommerce database.")

    # Read the CSV file into a DataFrame
    df_csv = pd.read_csv(csv_file_path)

    # Clean column names and handle null values
    df_csv = df_csv.where(pd.notnull(df_csv), None)
    df_csv.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df_csv.columns]

    # Create the table if it doesn't exist
    columns_csv = ', '.join([f'"{col}" {get_sql_type(df_csv[col].dtype)}' for col in df_csv.columns])
    create_table_query_csv = f'CREATE TABLE IF NOT EXISTS "{table_name_csv}" ({columns_csv})'
    cursor.execute(create_table_query_csv)
    print(f"Table {table_name_csv} created or already exists.")

    # Insert data into the table
    columns_str_csv = ', '.join([f'"{col}"' for col in df_csv.columns])
    placeholders_csv = ', '.join(['%s'] * len(df_csv.columns))
    insert_sql_csv = f'INSERT INTO "{table_name_csv}" ({columns_str_csv}) VALUES ({placeholders_csv})'
    data_tuples_csv = [tuple(row) for row in df_csv.to_numpy()]
    psycopg2.extras.execute_batch(cursor, insert_sql_csv, data_tuples_csv)

    conn.commit()
    print(f"Data from {csv_file_path} inserted successfully into {table_name_csv}.")

except psycopg2.Error as err:
    print(f"Error: {err}")
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    print("Database connection closed.")

Connected to the ECommerce database.
Table ecommerce_sales_10k created or already exists.
Data from ./ecommerce_sales_10k.csv inserted successfully into ecommerce_sales_10k.
Database connection closed.
